<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong>
       Importando bibliotecas
    </strong>
</p>

In [2]:
import pandas as pd
from openpyxl import Workbook, load_workbook
from openpyxl.styles import numbers
import xlwings as xw

<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
       Criando planilha de controle de entregas
    </strong>
</p>

In [8]:
# Criando variável com caminho para salvar arquivo
controle_path = "dados/controle_entregas.xlsx"

# Criamos um novo arquivo Excel vazio
wb = Workbook()

# Pegamos a aba ativa (primeira planilha do arquivo)
ws = wb.active

# Renomeamos a aba para ficar claro no projeto
ws.title = "Entregas"

# Criamos o cabeçalho principal da tabela
ws.append([
    "Data",                 # Data da entrega
    "Estado",               # UF da entrega
    "Motorista",            # Responsável pela rota
    "Entregas Previstas",   # Total esperado
    "Entregas no Prazo",    # Entregas dentro do prazo
    "Entregas Realizadas"   # Total concluído
])

# Inserimos alguns dados simulados realistas
ws.append(["01/01/2025", "SP", "Carlos", 120, 115, 110])
ws.append(["01/01/2025", "RJ", "Ana", 90, 85, 80])
ws.append(["01/01/2025", "MG", "João", 70, 65, 60])
ws.append(["02/01/2025", "SP", "Carlos", 100, 92, 95])
ws.append(["02/01/2025", "RJ", "Ana", 85, 70, 75])

# Salvamos o arquivo Excel na pasta dados
wb.save(controle_path)

print("Planilha criada com sucesso!")

Planilha criada com sucesso!


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
       Inserindo novos indicadores
    </strong>
</p>

In [ ]:
wb = load_workbook(controle_path)
ws = wb.active

ws["G1"] = "Taxa de sucesso"
ws["H1"] = "Taxa de atraso"

for linha in range(2, ws.max_row + 1):
        # G2              =   F2    /   D2    (Taxa de sucesso)
    ws[f'G{linha}'] = f'=F{linha} / D{linha}'

    #    H2               =1 - G2   (Taxa de atraso)
    ws[f'H{linha}'] = f'=1 - G{linha}'

wb.save(controle_path)
print('Fórmulas inseridas com sucesso!')

Fórmulas inseridas com sucesso!


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
       Formatando as taxas como percentual
    </strong>
</p>

In [10]:
wb = load_workbook(controle_path)
ws = wb.active

for linha in range(2, ws.max_row + 1):

    ws[f'G{linha}'].number_format = "0.00%"
    ws[f'H{linha}'].number_format = "0.00%"

wb.save(controle_path)
print('Tipagem alterada com sucesso')

Tipagem alterada com sucesso


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
        Resumo gerencial por estado
    </strong>
</p>

In [ ]:
df = pd.read_excel(controle_path, sheet_name="Entregas") 

df['Taxa de sucesso 2'] = df['Entregas Realizadas'] / df['Entregas Previstas'] 

resumo_estado = df.groupby('Estado').agg({   
    "Entregas Previstas" : "sum",
    "Entregas Realizadas" : "sum",
    "Taxa de sucesso" : "mean",
    "Taxa de atraso" :  "mean"
})

wb = load_workbook(controle_path)

if 'Resumo' in wb.sheetnames:
    del wb['Resumo']

ws_resumo = wb.create_sheet("Resumo")

ws_resumo.append(
    ["Estado", 
     'Entregas Previstas', 
     'Entregas Realizadas',
     'Taxa de sucesso',
     'Taxa de atraso' 
     ])

for estado, valores in resumo_estado.iterrows():
    ws_resumo.append([
        estado,
        valores['Entregas Previstas'],
        valores['Entregas Realizadas'],
        valores['Taxa de sucesso'],
        valores['Taxa de atraso']
    ])

for linha in range(2, ws_resumo.max_row + 1):
    ws_resumo[f'D{linha}'].number_format = '0.00%'
    ws_resumo[f'E{linha}'].number_format = '0.00%'


wb.save(controle_path)

<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
        Cálculos de taxas e inserindo no excel
    </strong>
</p>